---
title: Regularization
jupyter: python3
---

In [ ]:
#| echo: false
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib as mp
import sklearn
import statsmodels.api as sm
from sklearn import model_selection
from sklearn import metrics

import laUtilities as ut

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.regression.linear_model import OLS
import statsmodels.formula.api as smf

import warnings

np.random.seed(9876789)

In [ ]:
from statsmodels.datasets.longley import load_pandas
y = load_pandas().endog
X = load_pandas().exog
X['const'] = 1.0
X.index = X['YEAR']
y.index = X['YEAR']
X.drop('YEAR', axis = 1, inplace = True)
print(X.head())
print(y.head())

---

An important warning is issued stating the condition number is large. What does this mean?

In [ ]:
ols_model = sm.OLS(y, X)
ols_results = ols_model.fit()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    print(ols_results.summary())

## Condition Number

The notion of conditioning pertains to the perturbation behavior of a mathematical problem. A well-conditioned problem is one where small changes to the inputs produce small changes to the output. An ill-conditioned problem is one where small changes in the input can produce very large changes in the output. 

The condition number of a matrix provides an indication of how accurately you can compute with it. 

A large condition number tells us that our problem is ill conditioned, i.e., small changes to the input can produce very large changes in the output.

A small condition number tells us that our problem is well-conditioned.

---

The condition number is derived using matrix norms, which is beyond the scope of this course. However, we will use the following definition for the condition number of our matrix

$$
\kappa(X) = \frac{\sigma_{\text{max}}}{\sigma_{\text{min}}},
$$

where $\sigma_{\text{max}}$, $\sigma_{\text{min}}$ are the maximum and minimum singular values, respectively, of the design matrix $X$.

The SVD again provides us with important properties of a matrix.

---

Another important fact is that

$$
\kappa(X^TX) = \frac{\sigma_{\text{max}}^2}{\sigma_{\text{min}}^2}.
$$

This means that if we have a poorly conditioned data matrix $X$, then $X^TX$ is even more poorly conditioned.

This is why you should never work directly with $X^TX$ as it can be numerically unstable.

## Normal Equations

To solve the least-squares problem we solve the normal equations

$$
X^TX\boldsymbol{\beta} = X^Ty.
$$

These equations always have at least one solution. However, the *at least one* part is problematic.

If there are multiple solutions, they are in a sense all equivalent in that they yield the same value of $\Vert X\boldsymbol{\beta} - y\Vert$.

However, the actual values of $\boldsymbol{\beta}$ can vary tremendously and so it is not clear how best to interpret which solution is actually the best.

When does this problem occur? 

## Linear Dependence

It occurs when $X^TX$ is __not invertible.__

This happens when the columns of $X$ are linearly dependent -- that is, one column can be expressed as a linear combination of the other columns.

In that case, it is not possible to solve the normal equations by computing $\hat{\boldsymbol{\beta}} \neq (X^TX)^{-1}X^Ty.$

This is the simplest kind of __multicollinearity__.

---

What are the implications of a matrix not being invertible on the condition number?

If a matrix $Z = X^{T}X$ is not invertible, there is a zero singular value. This implies

$$
\kappa(Z) = \infty.
$$

In other words, the problem of solving an equation with a non-invertible matrix is completely ill-conditioned. In fact, it's a problem that is impossible to solve.

## Near Linear Dependence

Near linear dependence causes problems as well. This can happen, for example, due to  measurement errors. Or when two or more columns are __strongly correlated__.

In such a situation, we have some column of our design matrix that is __close to__ being a linear combination of the other columns.

When these situations occur we will have problems with linear regression.

---

As a result of near linear dependence, the smallest singular value of the design matrix $X$ will be close to zero. This means that $\kappa(X)$ will be very large.

The condition number tells us that a small change in the input to our problem can result in large changes to the output. 

This means that for a design matrix $X$ with near linearly dependent columns, the values we compute for $\boldsymbol{\beta}$ in our linear regression can vary significantly.

This is why we see the addition of a regularization (penalty) term involving $\boldsymbol{\beta}$ in the least squares minimization problem. This process *regularizes* the solution $\boldsymbol{\beta}$.


## Longley Dataset

Recall that the condition number of our data is around $10^8$. 

A large condition number is evidence of a problem.

As a general rule of thumb:

* If the condition number is less than 100, there is no serious problem
with multicollinearity.
* Condition numbers between 100 and 1000 imply moderate to strong multicollinearity.
* Condition numbers bigger than 1000 indicate severe multicollinearity.

---

Let's look at pairwise scatter plots of the Longley data.

In [ ]:
sns.pairplot(X[['GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP']])
plt.show()

We can see __very__ strong linear relationships between, e.g., __GNP Deflator__, __GNP__, and __Population.__

## Addressing Multicollinearity

Here are two strategies we can employ to address multicollinearity:

1. Ridge Regression
2. Model Selection via LASSO

::: {.aside}
PCA also addresses multicollinearity by transforming the correlated features into uncorrelated features. However in this approach you lose the original features, which is less explainable.
:::

## Ridge Regression

The first thing to note is that when columns of $X$ are nearly dependent, the components of $\hat{\boldsymbol{\beta}}$ tend to be __large in magnitude__.

:::: {.columns}
::: {.column width="50%"}

In [ ]:
#| fig-align: center
ax = ut.plotSetup(size=(4,2))
ut.centerAxes(ax)
u = np.array([1, 2])
v = np.array([4, 1])
alph = 1.6
beta = -1.25
sum_uv = (alph * u) + (beta * v)
ax.arrow(0, 0, u[0], u[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.arrow(0, 0, v[0], v[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.text(sum_uv[0]-.5, sum_uv[1]+0.25, r'$\mathbf{y}$',size=12)
ax.text(u[0]+0.25, u[1]-0.25, r'${\bf u}$', size=12)
ax.text(v[0]+0.25, v[1]+0.25, r'${\bf v}$',size=12)
ut.plotPoint(ax, sum_uv[0], sum_uv[1])
ax.plot(0, 0, '')
plt.show()

Consider a regression in which we are predicting the point $\mathbf{y}$ as a linear function of two $X$ columns, which we'll denote $\mathbf{u}$ and $\mathbf{v}$.
:::
::: {.column width="50%"}

In [ ]:
#| fig-align: center
ax = ut.plotSetup(size=(4, 2))
ut.centerAxes(ax)
u = np.array([1, 2])
v = np.array([4, 1])
alph = 1.6
beta = -1.25
sum_uv = (alph * u) + (beta * v)
ax.arrow(0, 0, u[0], u[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.arrow(0, 0, v[0], v[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.arrow(0, 0, alph * u[0], alph * u[1], head_width=0.2, 
         head_length=0.2, length_includes_head = True)
ax.arrow(alph * u[0], alph * u[1], sum_uv[0] - alph * u[0], sum_uv[1] - alph * u[1], 
         head_width=0.2, 
         head_length=0.2, length_includes_head = True, color = 'r')
ax.text(sum_uv[0]-2, sum_uv[1]+0.25, r'$\beta_1{\bf u}$+$\beta_2{\bf v}$',size=12)
ax.text(u[0]+0.25, u[1]-0.25, r'${\bf u}$', size=12)
ax.text(alph * u[0]+0.25, alph * u[1]-0.25, r'$\beta_1{\bf u}$', size=12)
ax.text(-2, 2.75, r'$\beta_2{\bf v}$', size=12)
ax.text(v[0]+0.25, v[1]+0.25, r'${\bf v}$',size=12)
ut.plotPoint(ax, sum_uv[0], sum_uv[1])
ax.plot(0, 0, '')
plt.show()

Via least-squares, we determine the coefficients $\beta_1$ and $\beta_2$.
:::
::::

---

Now consider if the columns of $X$ are __nearly dependent__.

:::: {.columns}
::: {.column width="50%"}

In [ ]:
#| fig-align: center
ax = ut.plotSetup(size=(4, 2))
ut.centerAxes(ax)
u = np.array([2, 1])
v = np.array([4, 1])
ax.arrow(0, 0, u[0], u[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.arrow(0, 0, v[0], v[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.text(sum_uv[0]-.5, sum_uv[1]+0.25, r'$\mathbf{y}$',size=12)
ax.text(u[0]+0.25, u[1]-0.25, r'${\bf u}$', size=12)
ax.text(v[0]+0.25, v[1]+0.25, r'${\bf v}$',size=12)
ut.plotPoint(ax, sum_uv[0], sum_uv[1])
ax.plot(0, 0, '')
plt.show()

:::
::: {.column width="50%"}

In [ ]:
#| fig-align: center
ax = ut.plotSetup(size=(4, 2))
ut.centerAxes(ax)
u = np.array([2, 1])
v = np.array([4, 1])
alph = 2.675
beta = -8.75
ax.arrow(0, 0, u[0], u[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.arrow(0, 0, v[0], v[1], head_width=0.2, head_length=0.2, length_includes_head = True)
ax.arrow(0, 0, alph * u[0], alph * u[1], head_width=0.2, 
         head_length=0.2, length_includes_head = True)
ax.arrow(alph * u[0], alph * u[1], sum_uv[0] - alph * u[0], sum_uv[1] - alph * u[1], 
         head_width=0.2, 
         head_length=0.2, length_includes_head = True, color = 'r')
ax.text(sum_uv[0]-2, sum_uv[1]+0.25, r'$\beta_1{\bf u}$+$\beta_2{\bf v}$',size=12)
ax.text(u[0]+0.25, u[1]-0.25, r'${\bf u}$', size=12)
ax.text(alph * u[0]+0.25, alph * u[1]-0.25, r'$\beta_1{\bf u}$', size=12)
ax.text(-2, 2.75, r'$\beta_2{\bf v}$', size=12)
ax.text(v[0]+0.25, v[1]+0.25, r'${\bf v}$',size=12)
ut.plotPoint(ax, sum_uv[0], sum_uv[1])
ax.plot(0, 0, '')
plt.show()

:::
::::

If you imagine the values of $\beta_1$ and $\beta_2$ necessary to create $\mathbf{y} = \beta_1{\bf u}$+$\beta_2{\bf v}$, you can see that $\beta_1$ and $\beta_2$ will be __very large__ in magnitude.

This geometric argument illustrates why the regression coefficients will be very large under multicollinearity.

As a result, the value of $\Vert\boldsymbol{\beta}\Vert_2$ will be very large.

## Ridge Regression

Ridge regression adjusts the least squares regression by shrinking the estimated coefficients towards zero.

The purpose is to fix the magnitude inflation of $\Vert\boldsymbol{\beta}\Vert_2$.

To do this, Ridge regression assumes that the model has no intercept term -- both the response and the predictors have been centered so that $\beta_0 = 0$.

Ridge regression then consists of adding a penalty term to the regression:

$$ 
\hat{\boldsymbol{\beta}} = \arg \min_\boldsymbol{\beta} \Vert X\boldsymbol{\beta} - y \Vert_2^2 + c\Vert\boldsymbol{\beta}\Vert_2^2.
$$

---

For any given $c$ this has a closed-form solution in which $\hat{\boldsymbol{\beta}} = (X^TX +cI)^{−1}X^T.$

The solution to the Ridge regression problem always exists and is unique, even when the data contains multicollinearity.

Here, $c \geq 0$ is a tradeoff parameter and controls the strength of the penalty term:

* When $c = 0$, we get the least squares estimator: $\hat{\boldsymbol{\beta}} = (X^TX)^{−1}X^T\mathbf{y}$
* When $c \rightarrow \infty$, we get $\hat{\boldsymbol{\beta}} \rightarrow 0.$
* Increasing the value of $c$ forces the norm of $\hat{\boldsymbol{\beta}}$ to decrease, yielding smaller coefficient estimates in magnitude.

For a finite, positive value of $c$, we are balancing two tasks: fitting
a linear model and shrinking the coefficients.

The coefficient $c$ is a __hyperparameter__ that controls the model complexity. We typically set $c$ by holding out data, i.e., __cross-validation.__

## Scaling

Note that the penalty term $\Vert\boldsymbol{\beta}\Vert_2^2$ would be unfair to the different predictors if they are not on the same scale. 

Therefore, if we know that the variables are not measured in the same units, we typically first perform unit normal scaling on the columns of $X$ and on $\mathbf{y}$ (to standardize the predictors), and then perform ridge regression.

Note that by scaling $\mathbf{y}$ to have zero-mean, we do not need (or include) an intercept in the model.

Another name for ridge regression is __Tikhanov regularization__. You may see this terminology used in textbooks on optimization.

::: {.aside}
Normalizing is needed for this specific method. This is in contrast to the previous lecture where we allowed the coefficients to correct for the scaling differences between different units of measure.
:::

---

Here is the performance of Ridge regression on the Longley data.

We are training on half of the data and using the other half for testing.

In [ ]:
#| fig-align: center
from sklearn.metrics import r2_score
nreps = 1000

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X[['GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP']])
y_std = scaler.fit_transform(y.values.reshape(-1, 1))

np.random.seed(1)

vals = []
for alpha in np.r_[np.array([0]), 10**np.linspace(-8.5, -0.5, 20)]:
    res = []
    for rep in range(nreps):
        X_train, X_test, y_train, y_test = model_selection.train_test_split(
            X_std, y_std,
            test_size=0.5)
        model = sm.OLS(y_train, X_train)
        results = model.fit_regularized(alpha = alpha, L1_wt = 0)
        y_oos_predict = results.predict(X_test)
        r2_test = r2_score(y_test, y_oos_predict)
        res.append(r2_test)
    vals.append([alpha, np.mean(res), np.std(res)/np.sqrt(nreps)])

results = np.array(vals)

In [ ]:
#| fig-align: center
ax = plt.figure(figsize = (6, 4)).add_subplot()
ax.errorbar(np.log10(results[1:][:, 0]), results[1:][:, 1], 
            results[1:][:, 2],
            label = 'Ridge Regression')
ax.hlines(results[0,1], np.log10(results[1, 0]), 
           np.log10(results[-1, 0]), linestyles = 'dashed',
          label = 'Without Regularization')
ax.hlines(results[0,1]+results[0,2], np.log10(results[1, 0]), 
           np.log10(results[-1, 0]), linestyles = 'dotted')
ax.hlines(results[0,1]-results[0,2], np.log10(results[1, 0]), 
           np.log10(results[-1, 0]), linestyles = 'dotted')
ax.tick_params(labelsize=12)
ax.set_ylabel('$R^2$', fontsize = 14)
plt.legend(loc = 'best')
ax.set_xlabel('$\\log_{10}(c)$', fontsize = 14)
ax.set_title('Ridge Regression Accuracy on Longley Data', fontsize = 16)
plt.show()

--- 

To sum up the idea behind Ridge regression: 

1. There may be many $\boldsymbol{\beta}$ values that are consistent with the equations.   
1. Over-fit $\boldsymbol{\beta}$ values tend to have large magnitudes.
1. We add the regularization term $c \Vert \boldsymbol{\beta}\Vert_2^2$ to the least squares to avoid those solutions.
1. We tune $c$ to an appropriate value via cross-validation.

## Model Selection

Of course, one might attack the problem of multicollinearity as follows:
    
- Multicollinearity occurs when variables (features) are close to linearly dependent.
- These variables do not contribute anything *meaningful* to the quality of the model
- As a result why not simply remove variables from the model that are nearly linearly dependent?

We create a new model when we remove these variables from our regression.

This strategy is called **model selection**.

---

One of the advantages of model selection is __interpretability__: by eliminating variables, we get a clearer picture of the relationship between truly useful features and dependent variables.

However, there is a big challenge inherent in model selection. In general, the possibilities to consider are exponential in the number of features.

That is, if we have $n$ features to consider, then there are $2^n-1$ possible models that incorporate one or more of those features. This space is usually too big to search directly.

Can we use Ridge regression for this problem?

:::: {.fragment}
Ridge regression does not set any coefficients exactly to zero unless $c\rightarrow \infty$, in which case they’re all zero. 

This means Ridge regression cannot perform variable selection. Even though it performs well in terms of prediction accuracy, it does not offer a clear interpretation.
::::


## The LASSO

LASSO differs from Ridge regression __only in terms of the norm__ used by the penalty term.

$$ 
\hat{\beta} = \arg \min_\beta \Vert X\beta - y \Vert_2^2 + c \Vert\beta\Vert_1.
$$

However, this small change in the norm makes a __big difference__ in practice.

The nature of the $\ell_1$ penalty will cause some coefficients to be shrunken to zero exactly.

This means that LASSO can perform model selection by telling us which variables to keep and which to set aside.

As $c$ increases, more coefficients are set to zero, i.e., fewer variables are selected.

In terms of prediction error, LASSO performs comparably to Ridge regression but it has a __big advantage with respect to interpretation.__

---

In [ ]:
#| fig-align: center
from sklearn.metrics import r2_score
nreps = 200

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X[['GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP']])
X_std = np.column_stack([X_std, np.ones(X_std.shape[0])])
y_std = scaler.fit_transform(y.values.reshape(-1, 1))

np.random.seed(1)

vals = []
mean_params = []
for alpha in np.r_[np.array([0]), 10**np.linspace(-5, -0.75, 10)]:
    res = []
    params = []
    for rep in range(nreps):
        X_train, X_test, y_train, y_test = model_selection.train_test_split(
            X_std, y_std,
            test_size=0.5)
        model = sm.OLS(y_train, X_train)
        results = model.fit_regularized(alpha = alpha, L1_wt = 1.0)
        y_oos_predict = results.predict(X_test)
        r2_test = r2_score(y_test, y_oos_predict)
        res.append(r2_test)
        params.append(results.params)
    vals.append([alpha, np.mean(res), np.std(res)/np.sqrt(nreps)])
    mean_params.append(np.r_[alpha, np.mean(params, axis = 0)])
results = np.array(vals)
mean_params = np.array(mean_params)

In [ ]:
#| fig-align: center
ax = plt.figure(figsize = (6, 4)).add_subplot()
ax.errorbar(np.log10(results[1:][:, 0]), results[1:][:, 1], 
            results[1:][:, 2],
            label = 'LASSO Regression')
ax.hlines(results[0,1], np.log10(results[1, 0]), 
           np.log10(results[-1, 0]), linestyles = 'dashed',
          label = 'Without Regularization')
ax.hlines(results[0,1]+results[0,2], np.log10(results[1, 0]), 
           np.log10(results[-1, 0]), linestyles = 'dotted')
ax.hlines(results[0,1]-results[0,2], np.log10(results[1, 0]), 
           np.log10(results[-1, 0]), linestyles = 'dotted')
ax.tick_params(labelsize=12)
ax.set_ylabel('$R^2$', fontsize = 14)
#ax.set_xlim([-4, -1])
plt.legend(loc = 'best')
ax.set_xlabel('$\\log_{10}(c)$', fontsize = 14)
ax.set_title('LASSO Accuracy on Longley Data', fontsize = 16)
plt.show()

---

In [ ]:
df = pd.DataFrame(mean_params, columns = ['$\log_{10}(c)$', 'GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP', 'const'])
param_df = df[['GNPDEFL', 'GNP', 'UNEMP', 'ARMED', 'POP', 'const']].iloc[1:].copy()
param_df.index = np.log10(df.iloc[1:]['$\log_{10}(c)$'])

In [ ]:
#| fig-align: center
param_df.plot()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size': 16})
plt.title('LASSO Coefficients vs $c$')
plt.show()

--- 

We can use the statsmodel `smf` sub-module to directly type formulas and expressions in the functions of the models. This allows us to, among other things,

- specify the name of the columns to be used to predict another column
- remove columns
- infer the type of the variable (e.g., categorical, numerical)
- apply functions to columns

The `smf` submodule makes use of the [patsy(https://patsy.readthedocs.io/en/latest/)] package. patsy is a Python package for describing statistical models (especially linear models, or models that have a linear component) and building design matrices. It is closely inspired by and compatible with the formula mini-language used in R and S.

In the following code cells we will see the syntax that is used to specify columns in the models and how to remove columns from our model

---

Here is an example where we specify the name of the columns to be used to predict another column.

In [ ]:
X['TOTEMP'] = y

In [ ]:
mod = smf.ols(formula='TOTEMP ~ GNPDEFL + GNP + UNEMP + ARMED + POP', data=X)
res = mod.fit()   
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    print(res.summary())

---

The formula

`formula='TOTEMP ~ GNPDEFL + GNP + UNEMP + ARMED + POP'`

is an R-style formula string that specifies the model.

The variable `TOTEMP` is the dependent variable, which is Total Employment in the Longley dataset.

The syntax `~` separates the dependent variable from the independent variables.

The sytnax `GNPDEFL + GNP + UNEMP + ARMED + POP` are the independent variables, which are GNP Deflator, Gross National Product, Number of Unemployed, Size of the Armed Forces, and Population.

---

This is an example where we remove columns from the data and exclude the y-intercept.

In [ ]:
mod = smf.ols(formula='TOTEMP ~ GNPDEFL + GNP + UNEMP - 1', data=X)
res = mod.fit()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    print(res.summary())

---

The formula is 

`formula='TOTEMP ~ GNPDEFL + GNP + UNEMP - 1'`

We still have the same dependent variable `TOTEMP`. The independent variables are
`GNPDEFL + GNP + UNEMP`

The syntax `-1` removes the intercept from the model. By default, an intercept is included in the model, but - 1 explicitly excludes it.

---

## The LASSO and Longley Data

Here are some of the important observations from using LASSO regression on the Longley dataset:

- We removed the near linearly dependent features from our model.
- We improved the condition number of the data by 4 orders of magnitude.
- There is only one variable whose condfidence interval contains 0.

## Flexible Modeling

To look at model selection in practice, we will consider another famous dataset.

The Guerry dataset is a collection of historical data used in support of Andre-Michel Guerry’s 1833 "Essay on the Moral Statistics of France."

>Andre-Michel Guerry’s (1833) Essai sur la Statistique Morale
de la France was one of the foundation studies of modern social science.
Guerry assembled data on crimes, suicides, literacy and other “moral
statistics,” and used tables and maps to analyze a variety of social issues
in perhaps the first comprehensive study relating such variables.

---

>Guerry’s results were startling for two reasons.
First he showed that rates of crime and suicide remained
remarkably stable over time, when broken
down by age, sex, region of France and even season
of the year; yet these numbers varied systematically
across departements of France. This regularity
of social numbers created the possibility to
conceive, for the first time, that human actions in
the social world were governed by social laws, just
as inanimate objects were governed by laws of the
physical world.

Source: "A.-M. Guerry’s Moral Statistics of France: Challenges for Multivariable
Spatial Analysis", Michael Friendly.  Statistical Science 2007, Vol. 22, No. 3, 368–399.

--- 

Here is the dataset.

In [ ]:
# Lottery is per-capital wager on Royal Lottery
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
df.head()

---

Here is a regression using the feature `Literacy`, `Wealth`, and `Region`.

In [ ]:
mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
res = mod.fit()
print(res.summary())

---

In the previous cell, using the patsy syntax determined that elements of `Region` were text strings, so it treated `Region` as a categorical variable. 

Alternatively, we could manually enforce this with the syntax on the following slide. Recall that the `-` sign is used to remove columns/variables. Here we remove the intercept from a model by.

---

In [ ]:
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) -1 ', data=df).fit()
print(res.summary())

---

We can also apply vectorized functions to the variables in our model. The following cell shows how to do this. In this case we apply the natural log function to the `Literacy` column and use this single column to predict the `Lottery` values.

In [ ]:
res = smf.ols(formula='Lottery ~ np.log(Literacy)', data=df).fit()
print(res.summary())

## Recap

We discussed how to perform regularization in linear regression to avoid issues of overfitting due to multicollinearity.

We discussed how the condition number of a matrix indicates whether we have issues with multicollinearity. 

We saw that large condition numbers indicate multicollinearity.

To address this issue we considered both Ridge and LASSO regression.

We also learned about the patsy syntax in the statsmodel package.